In [1]:
#first it decodes the message and writes it as a text file:
import whisper

model = whisper.load_model("tiny")  # Options: tiny, base, small, medium, large

result = model.transcribe("command_test.mp3")
print(result["text"])

100%|█████████████████████████████████████| 72.1M/72.1M [00:04<00:00, 16.2MiB/s]
C:\Users\vitya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 test test test I'm gonna be talking like this let's see how it handles when it's super loud and then also when I'm really close and it's super loud or when I'm pretty far away and it's kind of quiet let's see if it can handle a little bit of this a little bit of that some flaps and I think birthday party power off run function meeting at 5 p.m. in new Plaza Square you need a doctor's appointment pretty soon make sure you get that going don't forget to visit the birthday party at 7 p.m.


first lets see if it can do this in real time. 
to do this, since the model is trained on 30 second intervals we will need to break the audio up into 30 second chunks.
then while its transcribing one section, it will be recording the next section, that way its going to be at most 30 seconds behind.
it will save these 30 second chunks to a file, labeled with date and time and then it will search through the file for any new information


In [ ]:
#code words and new information
# first it will look throught the transcript for any new information.
# it will do this any time that there is a break in the transcription.
# every 5 minutes it will check through the transcript to see if there is a break with no new text
# if there isnt, then it will look through from the last time it checked, up to the current time.



In [ ]:
# ok so openai just costs literally nothing to use so i will just transcribe the audio, and
# then ask it to take all the events and stuff that it finds and return it as a csv file, 
# updating once a hour. 


In [136]:
import datetime
import re
import itertools


database = "Transcription.txt"

def remove_filler(text):
    fillers = [
        "uh", "um", "okay", "ok", "so", "actually", "anyways", "just", "basically", "kinda",
        "sort of", "I mean", "you know", "let me", "oh", "like", "gonna", "got to", "gotta",
        "wanna", "thanks", "thank you", "thanks for", "thanks for that", "right", "well",
        "you see", "I guess", "I think", "I suppose", "if you will", "I mean to say",
        "in other words", "to be honest", "to tell the truth", "frankly", "honestly", "seriously",
        "i think", "i guess", "i mean", "you know what i mean", "you know what i mean?",
        "you know", "you know?", "you know what i mean", "kind of", "sort of",
        "like i said", "like i was saying", "like i said before", "like i said earlier",
        "like i mentioned", "like i mentioned before", "like i mentioned earlier", "i feel like",
        "i feel like i need to", "i feel like i should", "i feel like i could",
        "i feel like i would", "i feel like i might", "i feel like i can",
        "i feel like i will", "i feel like i am", "i feel like i have to", "um",
        "uhm", "uh-huh", "yeah", "yep", "yes", "no", "nah", "nope","thanks for watching",
        "thanks for watching","thanks for watching",
        "right?", "isn't it?", "don't you think?", "you know what I mean?", "hello",
        "hi", "hey", "how are you?", "how's it going?", "how's everything?", "how's life?",
        "whats up?", "what's going on?", "what's new?", "what's happening?",
        "what's good?", "what's up with you?", "what's up with that?", "what's up with this?",
        "what's up with everything?", "what's up with life?", "what's up with the world?", "watching"
    ]

    # Sort by length to match longer phrases first

    fillers = sorted(fillers, key=len, reverse=True)
    for filler in fillers:

        # Remove filler word/phrase with optional leading/trailing spaces and punctuation

        text = re.sub(rf'(\s|^|[.,!?;:]){re.escape(filler)}(\s|[.,!?;:]|$)', ' ', text, flags=re.IGNORECASE)

    # Clean extra spaces and punctuation

    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s([.,!?;:])', r'\1', text)
    return text.strip()

datetime_now = datetime.datetime.now().strftime("%Y-%m-%d_%H")
# we only need to do this once ever hour so it uses less resources.
#cleaning step:
cleaned_file_name = (f"cleaned_transcription" + datetime_now + ".txt")

last_break = 1

with open(database, "r") as f:
    lines = f.readlines()
    count = 0
    for line in lines:
        count += 1
        if "--------------------" in line:
            last_break = count


with open(database, "r") as f:
    for line in itertools.islice(f, last_break, None):
        
        sentence = line.strip()

        #removes timestamp 

        sentence = re.sub(r'^\[\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\]\s*', '', sentence)
        sentence = remove_filler(sentence)

        if sentence == "": #if its empty, skip it
            continue
        
        
        with open(cleaned_file_name, "a") as cleaned_file:
            cleaned_file.write(sentence + "\n")
with open(database, "a") as f:
    f.write("-" * 20 + "\n")



In [ ]:
# api testing
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from datetime import datetime

with open(cleaned_file_name, "r") as f:\
    transcript = f.read()

#load api key
_ = load_dotenv(find_dotenv())
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
 
temperature = 0.3
variables = "event_name,event_date_start,event_date_end,start_time,end_time,event_location,important_details,people_attending"
response = openai.chat.completions.create(
    model = "gpt-4o",
    messages=[
        {"role": "system", "content": "You are an event extraction assistant."},
        {"role": "user", "content": "Extract all events from this transcript: \n" + transcript + 
        "\n and return it as a csv file formatted like this: \n" + variables + "\n" +
        "give just the csv, no filler text, with the dates being formatted as numbers like this: year-month-day (assume current year month and day unless otherwise specified) and times being formatted as hour:min \n"}
    ]
)
csv_content = response.choices[0].message.content

#saves to csv file for later use.
timestamp = datetime.now().strftime("%Y-%m-%d_%H")
filename = f"extraction_{timestamp}.csv"


with open(filename, 'w', encoding='utf-8') as f:
    f.write(csv_content)

print(f"CSV saved as: {filename}")
os.remove(cleaned_file_name)

CSV saved as: extraction_2025-07-21_16.csv


In [134]:
def clean_csv(filename):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.readlines()
    # Remove first and last lines 
    lines = lines[1:-1]
    with open(filename, "w", encoding="utf-8") as f:
        f.writelines(lines)

with open(filename, "r", encoding="utf-8") as f:
    first_line = f.readline().strip()
    print(f"First line: {first_line.strip()}")
    if first_line != variables:
        print("First line is not correct, cleaning CSV...")
        clean_csv(filename)

First line: ```csv
First line is not correct, cleaning CSV...


In [2]:
import pandas as pd
data = pd.read_csv("extraction_2025-08-01_15.csv")
data.head()

,event_name,event_date_start,event_date_end,start_time,end_time,event_location,important_details,people_attending
Birthday Party,2029-08-02,2029-08-02,NaN,NaN,NaN,It's going to be cold,bring the coat; Also I Transformers,Donov; Manay; Prathul; my brother
